In [1]:
# !pip install distfuse

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["TRANSFORMERS_CACHE"]="/workspace/cache"
os.environ["HF_DATASETS_CACHE"]="/workspace/cache"
os.environ["WANDB_DISABLED"] = "true"

In [10]:
import datasets
from datasets import load_dataset, Image
from distfuse import DistFuse
from tqdm import tqdm
import numpy as np
import pickle

In [3]:
model_checkpoints = [["sentence-transformers/LaBSE", "hf"]]
weights = [1]
dist_measure = "cosine" # cosine, euclidean, manhattan
model = DistFuse(model_checkpoints, weights, dist_measure=dist_measure, openai_token="", cohere_token="", device="cuda:0")

/root/anaconda3/envs/llava/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Load datasets

In [48]:
with open('/workspace/sea_vl/coyo_filtered_1k.pkl', 'rb') as file:   
    # Call load method to deserialze 
    coyo_dataset = pickle.load(file) 

In [49]:
for k in coyo_dataset.keys():
    for idx,data in enumerate(coyo_dataset[k]):
        if len(coyo_dataset[k][idx]) == 2:
            if len(coyo_dataset[k][idx][0]) > 1:
                coyo_dataset[k][idx] = coyo_dataset[k][idx][0]
        else:
            if len(coyo_dataset[k][idx]) > 1:
                coyo_dataset[k][idx] = coyo_dataset[k][idx]

In [4]:
# coyo_dataset = datasets.load_from_disk('/workspace/sea_vl/ALL_DATA_filtered')['text'][:]

In [6]:
sea_vqa = load_dataset('wit543/sea-vqa')
cvqa = load_dataset('afaji/cvqa')['test']

### Keywords extraction

In [76]:
keywords_cvqa = dict()
cvqa_country_question = dict()
cvqa_country_translated_question = dict()
count = 0
for data in tqdm(cvqa):
    language, country = [c for c in data['Subset'].split("'") if len(c)>2]
    if country not in keywords_cvqa:
        cvqa_country_question[country] = [data['Question']]
        cvqa_country_translated_question[country] = [data['Translated Question']]
        c1, c2, c3, c4 = data['Translated Options']
        keywords_cvqa[country] = [language,country,c1, c2, c3, c4]
    else:
        cvqa_country_question[country].append(data['Question'])
        cvqa_country_translated_question[country].append(data['Translated Question'])
        keywords_cvqa[country]+=[language,country,c1, c2, c3, c4]

100%|██████████████████████████████████████████████████████████████████████| 9752/9752 [04:46<00:00, 34.05it/s]


In [37]:
cvqa_country_question.keys()

dict_keys(['India', 'Ecuador', 'Spain', 'China', 'Uruguay', 'Indonesia', 'Kenya', 'Mexico', 'France', 'Pakistan', 'Romania', 'Russia', 'Singapore', 'South Korea', 'Mongolia', 'Argentina', 'Sri_Lanka', 'Rwanda', 'Malaysia', 'Nigeria', 'Bulgaria', 'Ireland', 'Ethiopia', 'Norway', 'Brazil', 'Japan', 'Philippines', 'Egypt', 'Colombia', 'Chile'])

In [41]:
coyo_dataset.keys()

dict_keys(['vn', 'my', 'sg', 'th', 'mm', 'id', 'la', 'ph', 'kh', 'tl', 'bn'])

### Keyword matching

In [83]:
coyo_dataset_full = datasets.load_from_disk('/workspace/sea_vl/ALL_DATA_filtered')

In [88]:
coyo_full_cat = dict()
for idx,data in tqdm(enumerate(coyo_dataset_full)):
    if data['short_url'] not in coyo_full_cat:
        coyo_full_cat[data['short_url']] = [data['text']]
    else:
        coyo_full_cat[data['short_url']].append(data['text'])

2704068it [11:47, 3821.90it/s]


In [107]:
text_keyword = dict()

list_country = [['id','Indonesia'],['my','Malaysia'],['sg','Singapore']]

for s_key, t_key in list_country:
    
    text_keyword[t_key] = []
    
    all_text_temp = []

    source = coyo_full_cat[s_key]
    target = list(set(keywords_cvqa[t_key]))

    for coyo in tqdm(source):
        for keyword in target:
            if keyword in coyo:
                all_text_temp.append(coyo)
                break
    text_keyword[t_key] += list(set(all_text_temp))

100%|██████████████████████████████████████████████████████████████| 658661/658661 [00:02<00:00, 278076.84it/s]


In [110]:
text_keyword['Malaysia'][:5]

['Know What Examples Of Unit Trust Companies In Malaysia And The Fund Names Itguy My',
 'Yingfa Y2000AF(M)-02 Mirrored Goggles | YingFa Ventures Malaysia',
 'PN10 and PN16 PP-R Pipes offered by PPR Pipe Fittings Malaysia',
 'Elderly Indian Malaysian husband and wife couple smiling and sitting close to each other',
 '7 Things you didn’t know about Malaysian actor Ashraf Sinclair']

In [111]:
text_keyword['Indonesia'][:5]

['FILE - In this Oct. 26, 2010 file photo, a worker rides a bicycle in front of the reactor building of the Bushehr nuclear power plant, just outside the southern city of Bushehr. Iran’s sole nuclear power plant has undergone a temporary emergency shutdown, state TV reported on Sunday, June 20, 2021. An official from the state electric energy company, Gholamali Rakhshanimehr, said on a talk show that the Bushehr plant shutdown began on Saturday and would last',
 'Indonesia Bali Villa Vacation Rentals In Canggu With Jacuzzi And Staff',
 'Sugar Made From Selected Cane, Processed With High Quality Technology Gunung Madu Sugar Premium Plantations Is Premium Quality Local Sugar. The Quality Of Sugar Is Routinely Tested And Certified By The Indonesian Sugar Plantation Research Center As A Lab',
 'Xxnamexx Mean In Indonesia Twitter',
 'Tuesday I’m (not) in the mood 😴 . . . . . #instadaily #livethelittlethings #abmhappylife #petitejoys #thatsdarling #darlingmovement #pursuepretty #thehappynow 

In [112]:
text_keyword['Singapore'][:5]

['Thirty-nine members of the Operation Lionheart contingent from the Singapore Civil Defence Force (SCDF), tasked with rescue efforts in quake-struck Padang in Indonesia, returned home on Oct 8, 2009. The SCDF will be be adding more men to Operation Li',
 'Chinese Sausages & Liver Sausages',
 'Forest Family Walk Singapore',
 'front sign of the ACRA entity which regulates accounting firms in Singapore',
 "Team Micromax Indian Aces' Ana Ivanovic (centre) of Serbia celebrates with teammates Sania Mirza (left) and Rohan Bopanna of India after their mixed doubles match against UAE Royals at the International Premier Tennis League (IPTL) in Singapore on De"]

### Semantic Search

In [113]:
top_k = 5

In [124]:
source = coyo_full_cat['my']
target = list(set(cvqa_country_translated_question['Malaysia']))

In [125]:
# CVQA
scores = model.score_pairs(source, target)

In [126]:
for idx,text in enumerate(source[:10]):
    sorted_index = np.argsort(scores[idx])[::-1]
    print(f"Text:{text}")
    for rank, cand_idx in enumerate(sorted_index[:top_k]):
        print(f"Candidate:{target[cand_idx]} score:{scores[idx][cand_idx]}")
    print("*"*100)

Text:Picture of VD069 - 36 Roses
Candidate:What is the name of flowers in the picture? score:0.4186636805534363
Candidate:What is the name of the flower in this picture? score:0.41344958543777466
Candidate:What is the name of the red fruit in this picture? score:0.38335657119750977
Candidate:What is the name of the yellow fruit in the picture? score:0.3673686981201172
Candidate:What is the occasion shown in this picture? score:0.3591980040073395
****************************************************************************************************
Text:Dell Swapable CD Drive DP/N HK718
Candidate:What is the main ingredients of the food/drink? score:0.18801964819431305
Candidate:The driver is travelling interstate to go home from Kuala Lumpur, which highway is he currently driving on? score:0.16853848099708557
Candidate:What religious festival is held here every year? score:0.16348205506801605
Candidate:In which island is the picture taken? score:0.15909041464328766
Candidate:When are the 

In [119]:
source = text_keyword['Malaysia']
target = list(set(cvqa_country_translated_question['Malaysia']))

In [120]:
# CVQA
scores = model.score_pairs(source, target)

In [121]:
for idx,text in enumerate(source[:10]):
    sorted_index = np.argsort(scores[idx])[::-1]
    print(f"Text:{text}")
    for rank, cand_idx in enumerate(sorted_index[:top_k]):
        print(f"Candidate:{target[cand_idx]} score:{scores[idx][cand_idx]}")
    print("*"*100)

Text:Know What Examples Of Unit Trust Companies In Malaysia And The Fund Names Itguy My
Candidate:Which building in Malaysia was named after the company? score:0.4393165409564972
Candidate:What is the name of Malaysian company that manufactures this car? score:0.392717182636261
Candidate:Most of similar houses can be found in which Malaysian state? score:0.38196417689323425
Candidate:What is the name of the red packets in Malaysia? score:0.35543930530548096
Candidate:In which of Phillipines, Malaysia, Thailand and Laos that the company does not operate? score:0.3463084101676941
****************************************************************************************************
Text:Yingfa Y2000AF(M)-02 Mirrored Goggles | YingFa Ventures Malaysia
Candidate:What is his role in Malaysia between year 2018 and 2020? score:0.18481595814228058
Candidate:In which of Indonesia, Malysia, Philippines and Vietnam where the weapon is not found? score:0.1646127551794052
Candidate:What is his role in